## Laboratorio #6  – Análisis de tráfico de red

### Objetivos
Aplicar los conocimientos sobre redes de computadoras en el análisis estadístico de tráfico de 
red, para detectar anomalías en el comportamiento de la red. 

### Desarrollo 
Para este ejercicio se utilizará el archivo analisis_paquetes.pcap, disponible en CANVAS. Trabaje el laboratorio en un jupyter notebook.

In [2]:
pip install scapy

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   --------------------- ------------------ 1.3/2.4 MB 8.4 MB/s eta 0:00:01
   -------------------------------------- - 2.4/2.4 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from scapy.all import *

¡Scapy se instaló correctamente!


In [4]:
from scapy.all import *
import pandas as pd

In [6]:
from scapy.all import *
print(conf.ifaces)

Source   Index  Name                                                                                    MAC                IPv4             IPv6                     
libpcap  1      Software Loopback Interface 1                                                                              127.0.0.1        ::1                      
libpcap  10     WAN Miniport (SSTP)                                                                                                                                  
libpcap  11     Microsoft Wi-Fi Direct Virtual Adapter #7                                               Intel:8c:a2:ae     169.254.107.82   fe80::2ba3:9966:f591:a48e
libpcap  12     Bluetooth Device (Personal Area Network) #3                                                                                                          
libpcap  13     WAN Miniport (PPTP)                                                                                                                                  
libp

In [7]:
# Usa el nombre EXACTO de la interfaz (cópialo de tu lista)
iface_correcta = "Intel(R) Wi-Fi 6 AX201 160MHz"

In [10]:
paquetes = sniff(count=25)  # ¡Sin especificar iface!

RuntimeError: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket or conf.L3socket6 to access layer 3

In [ ]:
# Verificar
print(f"Paquetes capturados: {len(paquetes)}")
print("Ejemplo del primer paquete:")
print(paquetes[0].summary())